
# Project Title: Custom ChatGPT App with Langchain

***Description***: The Official ChatGPT is great but need to be tailored as per the specific needs


# Advantages

- ***Increased Security***: Imply the security measures which are specific to organisation needs. i.e, control the access to the app via users perspective in which we can ensure of the confidental data and intectual property 

- ***Improved User Experience***: Customize the Layout, Branding, functionality to match the organizational needs. Add features like integration of customer relation management system or knowledge base 

- ***Great Flexibility***: New features and functionality as needed and not limited to prebuilt chatgpt app like we can make use of llama2 from huggingfacehub and integrate and modify app functionality accordingly to meet the customer specification let say to have a feature to track the progress of the customer or project  


### Dependencies

In [1]:
# %pip list

In [2]:
# %pip install -r ../LLM/requirements.txt -q

### Environmental Variables

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

try: 
    file = '../LLM/.env'
    keys = find_dotenv(file,raise_error_if_not_found=True)
    load_dotenv(keys, override=True)
    # print(os.environ.get('OPENAI_API_KEY'))
    print("Initialize Sucessfull!")
except:
    print("Need to Initialize Again...")

Initialize Sucessfull!


### Libraries

In [2]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.schema import SystemMessage

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory

from langchain_community.utilities import WikipediaAPIWrapper, DuckDuckGoSearchAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain.tools import DuckDuckGoSearchResults

from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent, create_openai_tools_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

from langchain import hub

### Initialize

In [3]:
# ChatBot
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1, streaming=True)

# History of Sessions
history = FileChatMessageHistory('./Sessions/CustomGPT_chat_history.json')


### Memory object

In [4]:
memory = ConversationBufferMemory(
    memory_key='chat_history', # chat memory will be stored in the chat history key, the key is used later in the chatprompttemplate
    chat_memory=history,
    return_messages=True # *necessary step which tells the memory to return the chat history as a list of messages instead of string
)

### Prompt

In [5]:
template = """
You are a Chatbot having a conservation with a Human. Answer the following as best as you can.
Questions: {q}
"""

prompt_template = PromptTemplate.from_template(template=template)
hubprompt = hub.pull('hwchase17/openai-tools-agent') # react base agent
# print(hubprompt.input_variables)
# print(hubprompt.template)

prompt = ChatPromptTemplate(
    input_variable=['content'],
    messages=[
        SystemMessage(content='You are a Chatbot having a conservation with a Human.'),
        MessagesPlaceholder(variable_name='chat_history'), # memory will be stored
        HumanMessagePromptTemplate.from_template('{content}')
    ]
)

### Chain

In [18]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)
# response = chain.invoke({'content':question})

### Tools

In [6]:
# 1. Python Repl
python_repl = PythonREPLTool()
python_repl_tool = Tool(name='Python_REPL', func=python_repl.run, description='Useful when you need to use python interpreter to answer a question. You should input Python code.')


# 2. Wikipedia 
wikipedia_api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_conten_chars_max=1500)
wikipedia = WikipediaQueryRun(api_wrapper=wikipedia_api_wrapper)
# wikipedia.invoke({'query': 'llamaindex'})
wikipedia_tool = Tool(name='Wikipedia', func=wikipedia.run, description='Useful when you need to look up a topic, etc on Wikipedia.')

# 3. DuckDuckGo
search_api_wrapper = DuckDuckGoSearchAPIWrapper(region='in-en', max_results=3, safesearch='moderate') # regions : https://serpapi.com/duckduckgo-regions
search = DuckDuckGoSearchResults(api_wrapper=search_api_wrapper)  # source='news'
# search.run('langchain')
duckduckgo_tool = Tool(name='DuckDuckGoSearch', func=search.run, description='Useful when you need to look up a recent update to date information on internet.')

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

# agent
agent = create_openai_tools_agent(llm, tools, hubprompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=False, handle_parsing_errors=True, max_iterations=10)

# Testing
# question = "Generate Steps to learn LLamaIndex and show me one use-case using python"
# response = agent_executor.invoke({'input': prompt_template.format(q=question)})
# response = agent_executor.invoke({"input": prompt_template.format(q=question)}) # , return_only_outputs=True
# print(response['input'])
# print(response['output'])

### User Interface

In [7]:
user = True

while user:
    question = input("Prompt: ") 
    
    if question in (' ', '', 'exit', 'quit', 'q', 'Q'):
        print("\nProcess End ...")
        break

    response = agent_executor.invoke({"input": question})
    for chunk in response['output']:
        print(chunk, end='', flush=True)
    print('\n\n')

Prompt:  what are the questions I have asked previously till so far? 


Here are the summaries of the topics you asked about:

1. Winston Churchill: Sir Winston Leonard Spencer Churchill was a British statesman, soldier, and writer who served as Prime Minister of the United Kingdom during World War II. He was known for his leadership and is considered one of the greatest prime ministers in British history.

2. "We Shall Fight on the Beaches": This speech was delivered by Winston Churchill to the House of Commons on 4 June 1940 during World War II. It was a declaration of resolve to continue the fight against Nazi Germany.

3. Battle of France: The Battle of France was the German invasion of France during World War II, leading to the fall of France and the occupation of certain regions by the Germans.

4. French Army during World War II: The French Army played a significant role during World War II, particularly in the Battle of France. The invasion by German forces led to the defeat and occupation of France.

5. Relationship between Winston Churchill and t

Prompt:  from the above questions which are meant to be most imp one?


The most important question among the ones you have asked so far could be the one about Winston Churchill's famous speech "We Shall Fight on the Beaches." This question delves into a significant moment in history during World War II and highlights Churchill's leadership and determination during a critical period. This speech is widely recognized and symbolizes the resilience and resolve of the British people in the face of adversity.




Prompt:  why it is so special and can you mention some bullet points


The "We Shall Fight on the Beaches" speech by Winston Churchill is considered special for several reasons. Here are some key points that make this speech significant:

1. Symbol of Determination: The speech symbolizes the unwavering determination of the British people to continue the fight against Nazi Germany despite facing great adversity.

2. Inspirational Tone: Churchill's powerful and inspirational words in the speech uplifted the morale of the British population and instilled a sense of resilience during a challenging time in World War II.

3. Defiance Against Aggression: The speech reflects Churchill's defiance against the aggression of the enemy and his commitment to resist and push back against the German forces.

4. Historic Context: The speech was delivered at a crucial moment when Britain stood alone against the German military advancement, making it a pivotal moment in British history.

5. Leadership Display: Churchill's leadership and strength in articulating a clear mess

Prompt:  thank you!


You're welcome! If you have any more questions in the future or need further assistance, feel free to ask. Have a great day!




Prompt:  q



Process End ...
